In [1]:
import pandas as pd
import torch

import os
import sys
sys.path.append(os.path.abspath('..'))
from scripts.training import run_epoch, train_loop
from utils import move_data_to_device, move_dict_to_cpu, plot_image, plot_image_boxes
from PsfSimulator import PsfDataset
from models.subpix_rcnn import SubpixRCNN
import matplotlib.pyplot as plt
import torch.nn as nn
from torchvision.ops import MultiScaleRoIAlign
from torchvision.ops import misc as misc_nn_ops
from torchvision.transforms._presets import ObjectDetection
from torchvision.models._api import register_model, Weights, WeightsEnum
from torchvision.models._meta import _COCO_CATEGORIES
from torchvision.models._utils import _ovewrite_value_param, handle_legacy_interface
from torchvision.models.resnet import resnet50, ResNet50_Weights
from torchvision.models.detection._utils import overwrite_eps
from torchvision.models.detection.backbone_utils import _resnet_fpn_extractor, _validate_trainable_layers
from torchvision.models.detection.faster_rcnn import _default_anchorgen, FasterRCNN, FastRCNNConvFCHead, RPNHead
from torchvision.models.detection.roi_heads import RoIHeads

from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
from torchvision.models.detection.generalized_rcnn import GeneralizedRCNN
import torch.nn.functional as F
import torch.optim as optim
import os
import datetime

from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
import torch

import sys, os
sys.path.append(os.path.abspath('..'))
from models.subpix_rcnn import SubpixRCNN
from utils import move_data_to_device, move_dict_to_cpu, plot_image_boxes, evaluate_predictions, evaluate_prediction
from PsfSimulator import PsfDataset

from scripts.plotting import PlotController
from scripts.evaluating import test_densities_fixed_snr, test_model_once_plot, evaluate_recall_per_snr, test_perlinnoise_fixed_snr
import numpy as np
import matplotlib.pyplot as plt
import pickle
from scipy.ndimage import gaussian_filter, gaussian_laplace, laplace

In [2]:
# Density 1 = 40 spots per 64x64 image
# Density 0.05 = 2 spots per 64x64 image

In [2]:
# Instatiate the dataset
seed = None

num_spots_min = 2
num_spots_max = 80
sigma_mean= 1.5
sigma_std = 0.1
snr_min = 2
snr_max = 20
snr_std = 0.0
base_noise_min = 300
base_noise_max = 10000
use_gauss_noise = False
gauss_noise_std = 0.0
use_perlin_noise = True
perlin_min_max = None#(0.4, 0.6)
img_w = 64
img_h = 64

train_dataset = PsfDataset(seed, 2000, num_spots_min, num_spots_max, sigma_mean, sigma_std,
                      snr_min, snr_max, snr_std, base_noise_min, base_noise_max, use_gauss_noise,
                      gauss_noise_std, use_perlin_noise, perlin_min_max, img_w, img_h)


valid_dataset = PsfDataset(seed, 100, num_spots_min, num_spots_max, sigma_mean, sigma_std,
                      snr_min, snr_max, snr_std, base_noise_min, base_noise_max, use_gauss_noise,
                      gauss_noise_std, use_perlin_noise, perlin_min_max, img_w, img_h)


# Print the number of samples in the training and validation datasets
print(pd.Series({
    'Training dataset size:': len(train_dataset),
    'Validation dataset size:': len(valid_dataset)}))

data_loader_params = {'batch_size':8,    'collate_fn': lambda batch: tuple(zip(*batch)),}

training_loader = torch.utils.data.DataLoader(train_dataset, **data_loader_params)
validation_loader = torch.utils.data.DataLoader(valid_dataset, **data_loader_params)

Training dataset size:      2000
Validation dataset size:     100
dtype: int64


In [3]:
def generate_checkpoint_path(model_name, project_name):

  # 1. Define the project directory within Colab's content area
  folder_path = os.path.join(os.curdir, project_name)

  # 2. Create the directory if it doesn't exist
  os.makedirs(folder_path, exist_ok=True)

  # 3. Generate a timestamped subdirectory
  timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
  checkpoint_dir = os.path.join(folder_path, timestamp)
  os.makedirs(checkpoint_dir, exist_ok=True)

  # 4. Construct the checkpoint path
  checkpoint_path = os.path.join(checkpoint_dir, f"{model_name}.pth")

  return checkpoint_path

In [4]:
from torch.utils.tensorboard import SummaryWriter

In [5]:
writer = SummaryWriter('runs/filters_perlin_std')

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

backbone = resnet_fpn_backbone("resnet50", weights=None, trainable_layers=5)
kwargs = {"nms_tresh": 0.1, "image_mean":[0,0,0], "image_std":[1,1,1], 'score_thresh': 0.5}

model = SubpixRCNN(backbone, num_classes=2, device=device, **kwargs)
model.to(device)
model.name = "testmodel"
optimizer = optim.Adam(model.parameters(), lr=0.0001)
path = r"D:\zeiss\Desktop\coding\Hilger\bachelor\notebooks\subpix_rcnn_models\2025-06-02_19-10-05\with_filters_long.pth"
model.load_state_dict(torch.load(path, map_location=device))

checkpoint_path = generate_checkpoint_path("with_filters_long", "subpix_rcnn_models")
num_epochs = 200

train_loop(model, training_loader, validation_loader, optimizer, device, num_epochs, checkpoint_path, writer)

cuda


c:\Users\zeiss\anaconda3\envs\hilger312\Lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'backbone_name' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


Custom SubpixRoIHeads successfully initialized!


Training Loop:   0%|          | 0/200 [00:00<?, ?it/s]

Train:   0%|          | 0/250 [00:00<?, ?it/s]

...

Eval:   0%|          | 0/13 [00:00<?, ?it/s]

Train:   0%|          | 0/250 [00:00<?, ?it/s]

Eval:   0%|          | 0/13 [00:00<?, ?it/s]

Train:   0%|          | 0/250 [00:00<?, ?it/s]

Eval:   0%|          | 0/13 [00:00<?, ?it/s]

# Now time to test the model

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

backbone = resnet_fpn_backbone("resnet50", weights=None, trainable_layers=5)
kwargs = {"nms_tresh": 0.1, "image_mean":[0,0,0], "image_std":[1,1,1], 'score_thresh': 0.7}

model = SubpixRCNN(backbone, num_classes=2, device=device, **kwargs)
model.to(device)
model.name = "testmodel"
optimizer = optim.Adam(model.parameters(), lr=0.0001)
path = r"D:\zeiss\Desktop\coding\Hilger\bachelor\notebooks\subpix_rcnn_models\2025-06-02_19-15-51\with_filters_long.pth"
model.load_state_dict(torch.load(path, map_location=device))
model.eval()

cuda
Custom SubpixRoIHeads successfully initialized!


SubpixRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0, 0, 0], std=[1, 1, 1])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=1e-05)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=1e-05)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=1e-05)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=1e-05)
          (relu): ReLU(inplace=True)
  

In [9]:
# Test densities at fixed SNRs
denseresults2snr = test_densities_fixed_snr(model, snr=2, num_images=300, device=device)
denseresults3snr = test_densities_fixed_snr(model, snr=3, num_images=300, device=device)
denseresults5snr = test_densities_fixed_snr(model, snr=5, num_images=300, device=device)
denseresults8snr = test_densities_fixed_snr(model, snr=8, num_images=300, device=device)

save_path = r'D:\zeiss\Desktop\coding\Hilger\bachelor\measurements\overnight_measurements\score07\d_2snr.pkl'
with open(save_path, 'wb') as f:
    pickle.dump(denseresults2snr, f)

save_path = r'D:\zeiss\Desktop\coding\Hilger\bachelor\measurements\overnight_measurements\score07\d_3snr.pkl'
with open(save_path, 'wb') as f:
    pickle.dump(denseresults3snr, f)

save_path = r'D:\zeiss\Desktop\coding\Hilger\bachelor\measurements\overnight_measurements\score07\d_5snr.pkl'
with open(save_path, 'wb') as f:
    pickle.dump(denseresults5snr, f)

save_path = r'D:\zeiss\Desktop\coding\Hilger\bachelor\measurements\overnight_measurements\score07\d_8snr.pkl'
with open(save_path, 'wb') as f:
    pickle.dump(denseresults8snr, f)

d:\zeiss\Desktop\coding\Hilger\bachelor\utils.py:86: RuntimeWarning: Mean of empty slice
  mse = np.nanmean(square_loc_errors)
c:\Users\zeiss\anaconda3\envs\hilger312\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:2019: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
d:\zeiss\Desktop\coding\Hilger\bachelor\utils.py:88: RuntimeWarning: Mean of empty slice
  me = np.nanmean(loc_errors)


Density: 0.04, Metrics: {'mean_precision': np.float64(0.305), 'mean_recall': np.float64(0.3566666666666667), 'mean_f1': np.float64(0.32166666666666666), 'mean_ji': np.float64(0.305), 'mean_loc_error': np.float64(0.35032894364660744), 'avg_std_loc_error': np.float64(0.0), 'mean_mean_squared_error': np.float64(0.13831975380346073), 'avg_std_mean_squared_error': np.float64(0.0), 'std_precision': np.float64(0.4327271400984433), 'std_recall': np.float64(0.47901519344959775), 'std_f1': np.float64(0.44237678761687105), 'std_ji': np.float64(0.4327271400984433), 'std_loc_error': np.float64(0.12485745891580873), 'std_mean_squared_error': np.float64(0.08517894212894771)}
Density: 0.05, Metrics: {'mean_precision': np.float64(0.4963888888888889), 'mean_recall': np.float64(0.4083333333333333), 'mean_f1': np.float64(0.43166666666666664), 'mean_ji': np.float64(0.35038888888888886), 'mean_loc_error': np.float64(0.33073603337702545), 'avg_std_loc_error': np.float64(0.024114152065498665), 'mean_mean_squa

d:\zeiss\Desktop\coding\Hilger\bachelor\PsfSimulator.py:130: RuntimeWarning: All-NaN slice encountered
  meanbg = np.nanmedian(array[y1:y2, x1:x2])


Density: 4.0, Metrics: {'mean_precision': np.float64(0.5639030540796177), 'mean_recall': np.float64(0.18249488752556237), 'mean_f1': np.float64(0.2749793357196671), 'mean_ji': np.float64(0.16019405027161493), 'mean_loc_error': np.float64(0.3367902536436996), 'avg_std_loc_error': np.float64(0.13730595517061792), 'mean_mean_squared_error': np.float64(0.133234153491272), 'avg_std_mean_squared_error': np.float64(0.09212858718898656), 'std_precision': np.float64(0.07073897080489164), 'std_recall': np.float64(0.03360773800825076), 'std_f1': np.float64(0.044884143088215235), 'std_ji': np.float64(0.03029121815088237), 'std_loc_error': np.float64(0.02718208168408644), 'std_mean_squared_error': np.float64(0.01854261081738823)}
Density: 0.04, Metrics: {'mean_precision': np.float64(0.9494444444444445), 'mean_recall': np.float64(0.99), 'mean_f1': np.float64(0.9627777777777777), 'mean_ji': np.float64(0.9494444444444445), 'mean_loc_error': np.float64(0.21085811725721532), 'avg_std_loc_error': np.floa

In [10]:
# Test recall per SNR
params = {'snr_min': 1, 'snr_max': 15}
recall_per_snr_iou05 = evaluate_recall_per_snr(model, 20000, device, 0.5, **params)
recall_per_snr_iou03 = evaluate_recall_per_snr(model, 20000, device, 0.3, **params)

save_path = r'D:\zeiss\Desktop\coding\Hilger\bachelor\measurements\overnight_measurements\score07\rec_iou05.pkl'
with open(save_path, 'wb') as f:
    pickle.dump(recall_per_snr_iou05, f)
save_path = r'D:\zeiss\Desktop\coding\Hilger\bachelor\measurements\overnight_measurements\score07\rec_iou03.pkl'
with open(save_path, 'wb') as f:
    pickle.dump(recall_per_snr_iou03, f)

Processing image 1/20000
Processing image 101/20000
Processing image 201/20000
Processing image 301/20000
Processing image 401/20000
Processing image 501/20000
Processing image 601/20000
Processing image 701/20000
Processing image 801/20000
Processing image 901/20000
Processing image 1001/20000
Processing image 1101/20000
Processing image 1201/20000
Processing image 1301/20000
Processing image 1401/20000
Processing image 1501/20000
Processing image 1601/20000
Processing image 1701/20000
Processing image 1801/20000
Processing image 1901/20000
Processing image 2001/20000
Processing image 2101/20000
Processing image 2201/20000
Processing image 2301/20000
Processing image 2401/20000
Processing image 2501/20000
Processing image 2601/20000
Processing image 2701/20000
Processing image 2801/20000
Processing image 2901/20000
Processing image 3001/20000
Processing image 3101/20000
Processing image 3201/20000
Processing image 3301/20000
Processing image 3401/20000
Processing image 3501/20000
Proc

AttributeError: Can't get local object 'evaluate_recall_per_snr.<locals>.<lambda>'

In [17]:
recall_per_snr_iou05= ((dict(recall_per_snr_iou05[0])), (dict(recall_per_snr_iou03[0])))
recall_per_snr_iou03 = ((dict(recall_per_snr_iou05[1])), (dict(recall_per_snr_iou03[1])))

save_path = r'D:\zeiss\Desktop\coding\Hilger\bachelor\measurements\overnight_measurements\score07\rec_iou05.pkl'
with open(save_path, 'wb') as f:
    pickle.dump(recall_per_snr_iou05, f)
save_path = r'D:\zeiss\Desktop\coding\Hilger\bachelor\measurements\overnight_measurements\score07\rec_iou03.pkl'
with open(save_path, 'wb') as f:
    pickle.dump(recall_per_snr_iou03, f)

In [13]:
print(type(recall_per_snr_iou05[0]))

<class 'collections.defaultdict'>


In [18]:
# Test noise levels
noisetest_snr3 = test_perlinnoise_fixed_snr(model, snr=3, num_images=300, device=device)
noisetest_snr2 = test_perlinnoise_fixed_snr(model, snr=2, num_images=300, device=device)
noisetest_snr5 = test_perlinnoise_fixed_snr(model, snr=5, num_images=300, device=device)
noisetest_snr8 = test_perlinnoise_fixed_snr(model, snr=8, num_images=300, device=device)

save_path = r'D:\zeiss\Desktop\coding\Hilger\bachelor\measurements\overnight_measurements\score07\noise_2snr.pkl'
with open(save_path, 'wb') as f:
    pickle.dump(noisetest_snr2, f)

save_path = r'D:\zeiss\Desktop\coding\Hilger\bachelor\measurements\overnight_measurements\score07\noise_3snr.pkl'
with open(save_path, 'wb') as f:
    pickle.dump(noisetest_snr3, f)

save_path = r'D:\zeiss\Desktop\coding\Hilger\bachelor\measurements\overnight_measurements\score07\noise_8snr.pkl'
with open(save_path, 'wb') as f:
    pickle.dump(noisetest_snr8, f)

save_path = r'D:\zeiss\Desktop\coding\Hilger\bachelor\measurements\overnight_measurements\score07\noise_5snr.pkl'
with open(save_path, 'wb') as f:
    pickle.dump(noisetest_snr5, f)

Noise level: (0.0, 1.0), Metrics: {'mean_precision': np.float64(0.7189239420423632), 'mean_recall': np.float64(0.5038333333333334), 'mean_f1': np.float64(0.5836725631809386), 'mean_ji': np.float64(0.42725633664279344), 'mean_loc_error': np.float64(0.3158263544418216), 'avg_std_loc_error': np.float64(0.12679505387331724), 'mean_mean_squared_error': np.float64(0.11891800515325993), 'avg_std_mean_squared_error': np.float64(0.08181932788471763), 'std_precision': np.float64(0.1275074076548825), 'std_recall': np.float64(0.15846231588474138), 'std_f1': np.float64(0.14892981472461306), 'std_ji': np.float64(0.14560566871654576), 'std_loc_error': np.float64(0.04772166827213649), 'std_mean_squared_error': np.float64(0.03013562081317901)}


d:\zeiss\Desktop\coding\Hilger\bachelor\PsfSimulator.py:123: RuntimeWarning: All-NaN slice encountered
  signal = np.nanmax(array[y1:y2, x1:x2])


Noise level: (0.05, 0.95), Metrics: {'mean_precision': np.float64(0.7186357586342107), 'mean_recall': np.float64(0.5206666666666666), 'mean_f1': np.float64(0.5974650079889634), 'mean_ji': np.float64(0.4391765359457763), 'mean_loc_error': np.float64(0.31274856718078614), 'avg_std_loc_error': np.float64(0.12841360214168812), 'mean_mean_squared_error': np.float64(0.11701028824223357), 'avg_std_mean_squared_error': np.float64(0.08171021488206678), 'std_precision': np.float64(0.12445414327503229), 'std_recall': np.float64(0.14562356341685306), 'std_f1': np.float64(0.1353193618325164), 'std_ji': np.float64(0.13787099663082733), 'std_loc_error': np.float64(0.04523122085230837), 'std_mean_squared_error': np.float64(0.02926970033429293)}
Noise level: (0.1, 0.9), Metrics: {'mean_precision': np.float64(0.7300342394104004), 'mean_recall': np.float64(0.5353333333333333), 'mean_f1': np.float64(0.6112351454093001), 'mean_ji': np.float64(0.4545074045071296), 'mean_loc_error': np.float64(0.313023269552

d:\zeiss\Desktop\coding\Hilger\bachelor\utils.py:86: RuntimeWarning: Mean of empty slice
  mse = np.nanmean(square_loc_errors)
c:\Users\zeiss\anaconda3\envs\hilger312\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:2019: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
d:\zeiss\Desktop\coding\Hilger\bachelor\utils.py:88: RuntimeWarning: Mean of empty slice
  me = np.nanmean(loc_errors)


Noise level: (0.0, 1.0), Metrics: {'mean_precision': np.float64(0.5188550986050986), 'mean_recall': np.float64(0.16233333333333333), 'mean_f1': np.float64(0.23806968947662435), 'mean_ji': np.float64(0.14136585267972077), 'mean_loc_error': np.float64(0.34186997617005793), 'avg_std_loc_error': np.float64(0.09060873741513943), 'mean_mean_squared_error': np.float64(0.1362017510329927), 'avg_std_mean_squared_error': np.float64(0.06093149433552126), 'std_precision': np.float64(0.24486487615689767), 'std_recall': np.float64(0.09856244495524426), 'std_f1': np.float64(0.12888420710445894), 'std_ji': np.float64(0.08577849517576194), 'std_loc_error': np.float64(0.08955054832187226), 'std_mean_squared_error': np.float64(0.06002643157495886)}
Noise level: (0.05, 0.95), Metrics: {'mean_precision': np.float64(0.536915915468547), 'mean_recall': np.float64(0.19466666666666665), 'mean_f1': np.float64(0.27708944938780244), 'mean_ji': np.float64(0.16687583565119796), 'mean_loc_error': np.float64(0.3437560

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

backbone = resnet_fpn_backbone("resnet50", weights=None, trainable_layers=5)
kwargs = {"nms_tresh": 0.1, "image_mean":[0,0,0], "image_std":[1,1,1], 'score_thresh': 0.67}

model = SubpixRCNN(backbone, num_classes=2, device=device, **kwargs)
model.to(device)
model.name = "testmodel"
optimizer = optim.Adam(model.parameters(), lr=0.0001)
path = r"D:\zeiss\Desktop\coding\Hilger\bachelor\notebooks\subpix_rcnn_models\2025-06-02_19-15-51\with_filters_long.pth"
model.load_state_dict(torch.load(path, map_location=device))
model.eval()

In [ ]:
# Test densities at fixed SNRs
denseresults2snr = test_densities_fixed_snr(model, snr=2, num_images=300, device=device)
denseresults3snr = test_densities_fixed_snr(model, snr=3, num_images=300, device=device)
denseresults5snr = test_densities_fixed_snr(model, snr=5, num_images=300, device=device)
denseresults8snr = test_densities_fixed_snr(model, snr=8, num_images=300, device=device)

save_path = r'D:\zeiss\Desktop\coding\Hilger\bachelor\measurements\overnight_measurements\score067\d_2snr.pkl'
with open(save_path, 'wb') as f:
    pickle.dump(denseresults2snr, f)

save_path = r'D:\zeiss\Desktop\coding\Hilger\bachelor\measurements\overnight_measurements\score067\d_3snr.pkl'
with open(save_path, 'wb') as f:
    pickle.dump(denseresults3snr, f)

save_path = r'D:\zeiss\Desktop\coding\Hilger\bachelor\measurements\overnight_measurements\score067\d_5snr.pkl'
with open(save_path, 'wb') as f:
    pickle.dump(denseresults5snr, f)

save_path = r'D:\zeiss\Desktop\coding\Hilger\bachelor\measurements\overnight_measurements\score067\d_8snr.pkl'
with open(save_path, 'wb') as f:
    pickle.dump(denseresults8snr, f)

In [ ]:
# Test recall per SNR
params = {'snr_min': 1, 'snr_max': 15}
recall_per_snr_iou05 = evaluate_recall_per_snr(model, 20000, device, 0.5, **params)
recall_per_snr_iou03 = evaluate_recall_per_snr(model, 20000, device, 0.3, **params)

save_path = r'D:\zeiss\Desktop\coding\Hilger\bachelor\measurements\overnight_measurements\score067\rec_iou05.pkl'
with open(save_path, 'wb') as f:
    pickle.dump(recall_per_snr_iou05, f)
save_path = r'D:\zeiss\Desktop\coding\Hilger\bachelor\measurements\overnight_measurements\score067\rec_iou03.pkl'
with open(save_path, 'wb') as f:
    pickle.dump(recall_per_snr_iou03, f)

In [ ]:
# Test noise levels
noisetest_snr3 = test_perlinnoise_fixed_snr(model, snr=3, num_images=300, device=device)
noisetest_snr2 = test_perlinnoise_fixed_snr(model, snr=2, num_images=300, device=device)
noisetest_snr5 = test_perlinnoise_fixed_snr(model, snr=5, num_images=300, device=device)
noisetest_snr8 = test_perlinnoise_fixed_snr(model, snr=8, num_images=300, device=device)

save_path = r'D:\zeiss\Desktop\coding\Hilger\bachelor\measurements\overnight_measurements\score067\noise_2snr.pkl'
with open(save_path, 'wb') as f:
    pickle.dump(noisetest_snr2, f)

save_path = r'D:\zeiss\Desktop\coding\Hilger\bachelor\measurements\overnight_measurements\score067\noise_3snr.pkl'
with open(save_path, 'wb') as f:
    pickle.dump(noisetest_snr3, f)

save_path = r'D:\zeiss\Desktop\coding\Hilger\bachelor\measurements\overnight_measurements\score067\noise_8snr.pkl'
with open(save_path, 'wb') as f:
    pickle.dump(noisetest_snr8, f)

save_path = r'D:\zeiss\Desktop\coding\Hilger\bachelor\measurements\overnight_measurements\score067\noise_5snr.pkl'
with open(save_path, 'wb') as f:
    pickle.dump(noisetest_snr5, f)

In [ ]:
# images = [] # FOR TESTING SCORE THRESHOLDS
# targets = []
# predictions = []
# model.eval()
# for i in range(50):
#     img, target = valid_dataset[i]
#     img = move_data_to_device(img, device)

#     images.append(img)
#     targets.append(target)
#     with torch.no_grad():
#         prediction = model([img])[0]
#     predictions.append(prediction)
# targets = move_data_to_device(targets, device)
# predictions = move_data_to_device(predictions, device)

In [ ]:
# evals = []
# for score in np.arange(0.5,0.95, 0.01):
#     print(f"Score {score}: {evaluate_predictions(predictions, targets, iou_thresh=0.5, score_thresh=score)}")

Score 0.5: {'mean_precision': np.float64(0.6305933985504183), 'mean_recall': np.float64(0.7850718286701257), 'mean_f1': np.float64(0.691228810908318), 'mean_ji': np.float64(0.5531333482177672), 'mean_loc_error': np.float64(0.28740319602197756), 'avg_std_loc_error': np.float64(0.1105305996507351), 'mean_mean_squared_error': np.float64(0.10067826104397275), 'avg_std_mean_squared_error': np.float64(0.06637076121398605), 'std_precision': np.float64(0.1831164911955375), 'std_recall': np.float64(0.1660587075171345), 'std_f1': np.float64(0.1636737428769076), 'std_ji': np.float64(0.20266108382530854), 'std_loc_error': np.float64(0.0651254860984144), 'std_mean_squared_error': np.float64(0.03714115002570901)}
Score 0.51: {'mean_precision': np.float64(0.6392457191587223), 'mean_recall': np.float64(0.7758001199866524), 'mean_f1': np.float64(0.6939217571955667), 'mean_ji': np.float64(0.5591636882312392), 'mean_loc_error': np.float64(0.28455278966966235), 'avg_std_loc_error': np.float64(0.1070299809

KeyboardInterrupt: 